In [ ]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.decomposition import PCA

import optuna
import optuna.visualization as vis

#### **Getting training & validation data**

In [ ]:
df_train = pd.read_csv('../databases/training.csv')
df_valid = pd.read_csv('../databases/validation.csv')

In [ ]:
X_train, y_train = df_train.drop(['IND_BOM_1_1'], axis=1), df_train['IND_BOM_1_1']
X_val, y_val = df_valid.drop(['IND_BOM_1_1'], axis=1), df_valid['IND_BOM_1_1']

#### **Parameters selection**

In [ ]:
params = {
    'hidden_layer_sizes': [32, 64, 128],
    'alpha': [0.0001, 0.0],
    'max_iter': [2000, 5000, 10000],
    'batch_size': [32, 64, 128],
    'activation': ['tanh', 'relu', 'sigmoid']
}

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    verbose=1,
    restore_best_weights=True
)

def objective(trial):
    
    model = keras.Sequential([
        keras.layers.Dense(
            units=trial.suggest_categorical(
                'hidden_layer_sizes',
                params['hidden_layer_sizes']
            ),
            activation=trial.suggest_categorical(
                'activation',
                params['activation']
            )
        ),
        keras.layers.Dense(
            units=trial.suggest_categorical(
                'hidden_layer_sizes',
                params['hidden_layer_sizes']
            ),
            activation=trial.suggest_categorical(
                'activation',
                params['activation']
            )
        ),
        keras.layers.Dense(units=1, activation='softmax')
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(
            trial.suggest_categorical(
                'alpha',
                params['alpha']
            )
        ),
        loss=keras.losses.BinaryCrossentropy(),
        metrics=['accuracy']
    )

    model.fit(
        X_train,
        y_train,
        batch_size=trial.suggest_categorical(
            'batch_size',
            params['batch_size']
        ),
        epochs=trial.suggest_categorical(
            'max_iter',
            params['max_iter']
        ),
        validation_data=(X_val, y_val),
        callbacks=[early_stopping]
    )

    loss, accuracy = model.evaluate(X_val, y_val)

    return accuracy

In [ ]:
n_trials = 15

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=n_trials)

In [ ]:
print('Best hyperparameters: ', study.best_params)
print('Best objective value: ', study.best_value)

#### **Saving study**

In [ ]:
import joblib

In [ ]:
save_path = './optuna_studies/mlp_study.pkl'

joblib.dump(study, save_path)